In [1]:
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
from torchvision import transforms as tf
from pytorch_lightning import Trainer
from CIFAR10Classifier import CIFAR10Classifier
from models.resnet import ResNet
import wandb
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import EarlyStopping

### Parameters and hyperparams

In [2]:
batch_size = 128
num_workers = 8
max_epochs = 100
is_fast_dev_run = False
is_wandb = True
model_name = "resnet" # "vgg" or "resnet"
activation = "relu" # "relu" or "leaky"

In [3]:
if is_wandb:
    logger = WandbLogger(project="cifar10", name=f"{model_name}_{activation}")
else:
    logger = None

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: vetitk. Use `wandb login --relogin` to force relogin


In [4]:
transform = tf.Compose([tf.ToTensor(), tf.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

cifar_train = CIFAR10(root="data", train=True, download=True, transform=transform)
cifar_val = CIFAR10(root="data", train=False, download=True, transform=transform)

train_loader = DataLoader(cifar_train, batch_size=batch_size, shuffle=True, num_workers=num_workers)
val_loader = DataLoader(cifar_val, batch_size=batch_size, shuffle=False, num_workers=num_workers)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
model = CIFAR10Classifier(model=ResNet())
trainer = Trainer(max_epochs=max_epochs,
                  gpus=0,
                  fast_dev_run=is_fast_dev_run,
                  logger=logger,
                  callbacks=[EarlyStopping(monitor="val_loss", patience=5)])


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [6]:
trainer.fit(model, train_loader, val_loader)


  | Name  | Type             | Params
-------------------------------------------
0 | model | ResNet           | 632 K 
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
632 K     Trainable params
0         Non-trainable params
632 K     Total params
2.529     Total estimated model params size (MB)


Epoch 0:  33%|███▎      | 156/470 [17:54<36:02,  6.89s/it, loss=1.44, v_num=p3q6]

/Users/vetitk/opt/anaconda3/envs/ml_dev/lib/python3.10/site-packages/pytorch_lightning/trainer/trainer.py:724: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
